# Summarize files and folders in AWS

This notebook provides an example of how to list files in an AWS bucket (including a prefix search) and group according to folder definitions

In [4]:
import boto3
import urllib3

import pandas as pd

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
bucket = "wbg-geography01"
prefix = "sylvera"
region = "us-east-1"
s3client = boto3.client("s3", region_name=region, verify=False)

# Loop through the S3 bucket and get all the file keys
more_results = True
try:
    del token  # noqa
except Exception:
    pass
loops = 0

all_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket,
            ContinuationToken=token,  # noqa
            Prefix=prefix,  # noqa
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        all_res.append(res)

Completed loop: 0
Completed loop: 1
Completed loop: 2
Completed loop: 3
Completed loop: 4
Completed loop: 5
Completed loop: 6
Completed loop: 7
Completed loop: 8
Completed loop: 9
Completed loop: 10
Completed loop: 11
Completed loop: 12
Completed loop: 13
Completed loop: 14
Completed loop: 15
Completed loop: 16
Completed loop: 17
Completed loop: 18
Completed loop: 19
Completed loop: 20
Completed loop: 21
Completed loop: 22
Completed loop: 23
Completed loop: 24
Completed loop: 25
Completed loop: 26
Completed loop: 27
Completed loop: 28
Completed loop: 29
Completed loop: 30
Completed loop: 31
Completed loop: 32
Completed loop: 33
Completed loop: 34
Completed loop: 35
Completed loop: 36
Completed loop: 37
Completed loop: 38
Completed loop: 39
Completed loop: 40
Completed loop: 41
Completed loop: 42
Completed loop: 43
Completed loop: 44
Completed loop: 45
Completed loop: 46
Completed loop: 47
Completed loop: 48
Completed loop: 49
Completed loop: 50
Completed loop: 51
Completed loop: 52
Com

In [11]:
inD = pd.DataFrame(all_res)
inD["folder"] = inD["Key"].apply(lambda x: "_".join(x.split("/")[:]))

,Key,LastModified,ETag,Size,StorageClass
0,.DS_Store,2020-09-24 18:15:42+00:00,"""098b7ee247e2688d3c110358e95be940""",24580,STANDARD
1,AIS/,2021-06-17 16:09:49+00:00,"""bd752504006b667e1139c9383472e928""",0,INTELLIGENT_TIERING
2,AIS_outputs/205762000.geojson,2021-11-30 23:13:40+00:00,"""36406a0f5b1d20b5f734ef775feb8b39""",630475,STANDARD
3,AIS_outputs/219833000.geojson,2021-11-30 23:13:40+00:00,"""e049985ba5d6327f1aafb33e8e92707a""",355694,STANDARD
4,AIS_outputs/229380000.geojson,2021-11-30 23:13:40+00:00,"""fb6c2da55db0cf1e7022d7d04c8de236""",539934,STANDARD
...,...,...,...,...,...
65246,sylvera/MSL/0007MZ/UAV-LS/GIL06/L3A/0007MZ_GIL...,2022-12-27 12:13:45+00:00,"""cc1d2ab954a43f1fda2902eb547703aa-4""",28143740,STANDARD
65247,sylvera/MSL/0007MZ/UAV-LS/GIL06/L3A/0007MZ_GIL...,2022-12-27 12:13:44+00:00,"""9fbaf7579c5c7ec7cc263bc161da743c-2""",14073598,STANDARD
65248,sylvera/MSL/0007MZ/UAV-LS/GIL06/L3A/0007MZ_GIL...,2022-12-27 12:13:46+00:00,"""9dfcc537eabdc7c09ea4fd687c024787""",1972843,STANDARD
65249,sylvera/MSL/0007MZ/UAV-LS/GIL06/L3B/0007MZ_GIL...,2022-12-27 13:16:38+00:00,"""0c2ad1ebb1ed02bf783a042d6bcc3308-84""",703036683,STANDARD
